In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('201901-202003.csv')
data = data.fillna('')
data04 = pd.read_csv('202004.csv')

In [27]:
df = data.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']).AMT.sum().reset_index(drop=False)
df04 = data04.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']).AMT.sum().reset_index(drop=False)

In [28]:
df07 = df.loc[df['REG_YYMM']==201907]
df1904 = df.loc[df['REG_YYMM']==201904]
df2004 = df04
covid = pd.concat([df1904.iloc[:,1:3],df2004.iloc[:,1:3]],axis=0).drop_duplicates(['CARD_SIDO_NM','STD_CLSS_NM'],keep=False).STD_CLSS_NM.unique()

In [29]:
eval04 = pd.concat([df07.iloc[:,1:3],df04.iloc[:,1:3]],axis=0).drop_duplicates(['CARD_SIDO_NM','STD_CLSS_NM'],keep=False)

In [30]:
test04 = pd.concat([df04.iloc[:,1:3],eval04],axis=0).drop_duplicates(['CARD_SIDO_NM','STD_CLSS_NM'],keep=False)

In [33]:
test04['REG_YYMM'] = 202007
test04= test[['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']]

In [36]:
df['covid'] = 1
for virus in covid:
    df.loc[(df['STD_CLSS_NM'] == virus),'covid'] = 0
    
test04['covid'] = 1
for virus in covid:
    test04.loc[(test04['STD_CLSS_NM'] == virus),'covid'] = 0

In [40]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

dtypes = test04.dtypes
encoders = {}
for column in test04.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(test04[column])
        encoders[column] = encoder
        
test04_num = test04.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    test04_num[column] = encoder.transform(test04[column])

In [286]:
test04_num
#new_val

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,covid,AMT
0,202007,0,0,1,1.064131e+08
1,202007,0,1,1,2.203884e+09
2,202007,0,2,1,1.196770e+09
3,202007,0,3,0,1.674177e+07
4,202007,0,5,0,1.546912e+06
...,...,...,...,...,...
4378,202007,15,28,0,3.701397e+05
4380,202007,15,30,0,9.241408e+05
4385,202007,15,35,0,1.812219e+05
4404,202007,16,15,0,3.448145e+04


In [264]:
df1903 = df.loc[df['REG_YYMM']==201903]
print((len(df04)-len(df1903))/len(df1903) * 100)
print((len(df07)-len(df1903))/len(df07)*100)
print(len(df07) * -3.17/100)
print(len(df07) * 1.4/100)
new_val = len(df07) + (-20 + 9)

-3.1746031746031744
1.4084507042253522
-20.2563
8.946


In [65]:
df07

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
3790,201907,강원,건강보조식품 소매업,92505731
3791,201907,강원,골프장 운영업,5032904403
3792,201907,강원,과실 및 채소 소매업,953346915
3793,201907,강원,관광 민예품 및 선물용품 소매업,29203800
3794,201907,강원,그외 기타 스포츠시설 운영업,6841100
...,...,...,...,...
4424,201907,충북,피자 햄버거 샌드위치 및 유사 음식점업,1579520158
4425,201907,충북,한식 음식점업,23535980210
4426,201907,충북,호텔업,25868500
4427,201907,충북,화장품 및 방향제 소매업,599495492


In [287]:
train_num = df_num.sample(frac=1, random_state=0)
X = train_num.drop(['AMT'], axis=1)
Y = np.log1p(train_num['AMT'])

# 모델

In [323]:
import xgboost as xgb
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [298]:
def rmsle_1(y_pred, data):
    y_true = np.array(data.get_label())
    score= np.sqrt(np.square(np.log1p(y_pred + 1) - np.log1p(y_true + 1)).mean())
    return 'rmsle', score, False

def rmsle(y_pred, data): 
    y_true = np.array(data)
    score= np.sqrt(np.square(np.log1p(y_pred + 1) - np.log1p(y_true + 1)).mean())
    return print(score)

In [316]:
def lgb_model(N,test,X,Y):
    pred = 0
    X_train, X_test, y_train, y_test = train_test_split( X, Y, 
                                          test_size=0.2, random_state=101)
    for i in range(N):
        train_lds = lgb.Dataset(X_train, label=y_train)
        val_lds = lgb.Dataset(X_test, label=y_test)
        params = {
                'max_depth': -1,
                'learning_rate' : 0.05,
                'boosting_type': 'gbdt',
                'objective': 'regression',
                'tweedie_variance_power': 1.1,
                'metric': 'custom',
                'sub_row' : 0.75,
                'lambda_l2' : 0.1,
                'seed': 101*i
            }
        model = lgb.train(params,
                      train_lds,
                      10000,
                      val_lds,
                      verbose_eval = 100,
                      early_stopping_rounds = 100,
                      feval=rmsle_1
                     )
        pred += np.expm1(model.predict(test))
    pred /= N
    return pred

In [313]:
def xgb_model(N,test,X,Y):
    pred = 0
    X_train, X_test, y_train, y_test = train_test_split( X, Y, 
                                          test_size=0.2, random_state=101)
    for i in range(N):
        train_xds = xgb.DMatrix(X_train, label=y_train)
        val_xds = xgb.DMatrix(X_test, label=y_test)
        watchlist = [(val_xds, 'eval'), (train_xds, 'train')]
        params = {
            'max_depth': 3,
            'learning_rate' : 0.05,
            'objective': 'reg:tweedie',
            'loss_function': 'rmse',
            'eta' : 1,
            'lambda_l2' : 0.1
            }
        test = xgb.DMatrix(test)
        model = xgb.train(params,train_xds,10000,watchlist,verbose_eval = 100,early_stopping_rounds = 100)
        
        pred += np.expm1(model.predict(test))
    pred /= N
    return pred

In [301]:
test04_num

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,covid,AMT
0,202007,0,0,1,1.064131e+08
1,202007,0,1,1,2.203884e+09
2,202007,0,2,1,1.196770e+09
3,202007,0,3,0,1.674177e+07
4,202007,0,5,0,1.546912e+06
...,...,...,...,...,...
4378,202007,15,28,0,3.701397e+05
4380,202007,15,30,0,9.241408e+05
4385,202007,15,35,0,1.812219e+05
4404,202007,16,15,0,3.448145e+04


In [371]:
#catb =cat_model(1,test04_num.iloc)
xgb=xgb_model(2,test04_num.iloc[:,:-1],X,Y)
lgb=lgb_model(2,test04_num.iloc[:,:-1],X,Y)

AttributeError: 'numpy.ndarray' object has no attribute 'DMatrix'

In [320]:
pred = lgb * 0.7 + xgb * 0.3
test04_num['AMT'] = pred

C:\Users\s_m04\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [321]:
final = test04_num.sort_values(by=['AMT'], axis=0)[::-1][:new_val]

In [324]:
xgb=xgb_model(1,final.iloc[:,:-1],X,Y)
lgb=lgb_model(1,final.iloc[:,:-1],X,Y)
pred = lgb * 0.7 + xgb * 0.3
final['AMT'] = pred

[17:06:52] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { lambda_l2, loss_function } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	eval-tweedie-nloglik@1.5:56.17811	train-tweedie-nloglik@1.5:55.82787
Multiple eval metrics have been passed: 'train-tweedie-nloglik@1.5' will be used for early stopping.

Will train until train-tweedie-nloglik@1.5 hasn't improved in 100 rounds.
[100]	eval-tweedie-nloglik@1.5:17.98151	train-tweedie-nloglik@1.5:17.91942
[200]	eval-tweedie-nloglik@1.5:17.97418	train-tweedie-nloglik@1.5:17.91258
[300]	eval-tweedie-nloglik@1.5:17.97212	train-tweedie-nloglik@1.5:17.91067
[400]	eval-tweedie-nloglik@1.5:17.97116	train-tweedie-nloglik@1.5:17.90971
[500]	eval-tweedie-nloglik@1.5:17.97050	train-twe

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmsle: 0.0454274
[200]	valid_0's rmsle: 0.0386236
[300]	valid_0's rmsle: 0.0360648
[400]	valid_0's rmsle: 0.0349031
[500]	valid_0's rmsle: 0.0336929
[600]	valid_0's rmsle: 0.0325723
[700]	valid_0's rmsle: 0.031443
[800]	valid_0's rmsle: 0.0304554
[900]	valid_0's rmsle: 0.0298639
[1000]	valid_0's rmsle: 0.029163
[1100]	valid_0's rmsle: 0.0282457
[1200]	valid_0's rmsle: 0.0279775
[1300]	valid_0's rmsle: 0.0275836
[1400]	valid_0's rmsle: 0.0270831
[1500]	valid_0's rmsle: 0.0267875
[1600]	valid_0's rmsle: 0.0265814
[1700]	valid_0's rmsle: 0.0260058
[1800]	valid_0's rmsle: 0.0257331
[1900]	valid_0's rmsle: 0.0255424
[2000]	valid_0's rmsle: 0.0253843
[2100]	valid_0's rmsle: 0.0252084
[2200]	valid_0's rmsle: 0.0251578
[2300]	valid_0's rmsle: 0.0251195
[2400]	valid_0's rmsle: 0.0250811
[2500]	valid_0's rmsle: 0.0249693
[2600]	valid_0's rmsle: 0.0247602
[2700]	valid_0's rmsle: 0.0246619
[2800]	valid_0's rmsle: 0.02458

In [326]:
final
final['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(final['CARD_SIDO_NM'])
final['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(final['STD_CLSS_NM'])

In [327]:
final

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,covid,AMT
322,202007,서울,한식 음식점업,1,1.603303e+11
71,202007,경기,한식 음식점업,1,1.534575e+11
43,202007,경기,기타 대형 종합 소매업,1,1.402950e+11
67,202007,경기,차량용 주유소 운영업,1,1.293033e+11
55,202007,경기,슈퍼마켓,1,1.285046e+11
...,...,...,...,...,...
221,202007,대전,그외 기타 스포츠시설 운영업,0,2.261178e+05
3977,202007,광주,택시 운송업,0,2.201503e+05
121,202007,경북,기타 수상오락 서비스업,0,1.996942e+05
372,202007,울산,여행사업,0,1.926772e+05


# 최종 합치기

In [353]:
answer04 = pd.read_csv('202004.csv')
answer04 = answer04.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']).AMT.sum().reset_index(drop=False)
submission04 = pd.read_csv('submission.csv', index_col=0)
#submission04 = submission04.loc[submission04['REG_YYMM']==202004]
#submission04 = submission04[['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM']]
#submission04 = submission04.merge(answer04, left_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
#submission04 = submission04.fillna(0)

In [357]:
submission04 = submission04.loc[submission04['REG_YYMM']==202004]
submission04 = submission04[['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM']]
submission04 = submission04.merge(answer04, left_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission04 = submission04.fillna(0)
submission04

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,8.882399e+07
1,202004,강원,골프장 운영업,4.708347e+09
2,202004,강원,과실 및 채소 소매업,1.121029e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.436078e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
692,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.373636e+09
693,202004,충북,한식 음식점업,1.891104e+10
694,202004,충북,호텔업,1.412150e+07
695,202004,충북,화장품 및 방향제 소매업,4.505074e+08


In [366]:
submission07 = pd.read_csv('submission.csv', index_col=0)
submission07 = submission07.loc[submission07['REG_YYMM']==202007]
submission07 = submission07[['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM']]
submission07 = submission07.merge(final, left_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission07 = submission07.fillna(0)
submission07

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202007,강원,건강보조식품 소매업,1.463780e+08
1,202007,강원,골프장 운영업,2.488717e+09
2,202007,강원,과실 및 채소 소매업,1.240494e+09
3,202007,강원,관광 민예품 및 선물용품 소매업,2.953535e+07
4,202007,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
692,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.646251e+09
693,202007,충북,한식 음식점업,1.575154e+10
694,202007,충북,호텔업,1.123549e+07
695,202007,충북,화장품 및 방향제 소매업,4.298112e+08


In [365]:
final = final.drop(['covid'],axis=1)

In [359]:
final

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,covid,AMT
322,202007,서울,한식 음식점업,1,1.603303e+11
71,202007,경기,한식 음식점업,1,1.534575e+11
43,202007,경기,기타 대형 종합 소매업,1,1.402950e+11
67,202007,경기,차량용 주유소 운영업,1,1.293033e+11
55,202007,경기,슈퍼마켓,1,1.285046e+11
...,...,...,...,...,...
221,202007,대전,그외 기타 스포츠시설 운영업,0,2.261178e+05
3977,202007,광주,택시 운송업,0,2.201503e+05
121,202007,경북,기타 수상오락 서비스업,0,1.996942e+05
372,202007,울산,여행사업,0,1.926772e+05


In [367]:
submission = pd.concat([submission04,submission07],axis=0)
submission.to_csv('final.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,8.882399e+07
1,202004,강원,골프장 운영업,4.708347e+09
2,202004,강원,과실 및 채소 소매업,1.121029e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.436078e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00


In [373]:
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,8.882399e+07
1,202004,강원,골프장 운영업,4.708347e+09
2,202004,강원,과실 및 채소 소매업,1.121029e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.436078e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
692,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.646251e+09
693,202007,충북,한식 음식점업,1.575154e+10
694,202007,충북,호텔업,1.123549e+07
695,202007,충북,화장품 및 방향제 소매업,4.298112e+08


In [376]:
obama = pd.read_csv('submission.csv', index_col=0)
obama.AMT 

id
0       8.236446e+10
1       5.794562e+10
2       2.155375e+10
3       1.641869e+10
4       2.376972e+10
            ...     
1389    6.101351e+09
1390    2.282981e+10
1391    3.065684e+10
1392    3.226583e+10
1393    4.200472e+10
Name: AMT, Length: 1394, dtype: float64

In [387]:
obama.AMT = submission.AMT.values

In [390]:
obama.to_csv('final.csv', encoding='utf-8-sig')